-----------------------
## 1) New way of scraping  11.4.2024
-----------------------

In [1]:
from run import Runner
runner = Runner()

## Complete scraping + updating DB

In [7]:
runner.run_complete_scraping(["prodej_byty", 
                              "prodej_domy",
                              "prodej_pozemky", 
                              "prodej_komercni",
                              "prodej_ostatni",
                              "pronajem_byty",
                              "pronajem_domy"])

SCRAPING Prices for https://www.sreality.cz/api/cs/v2/estates?category_type_cb=1&category_main_cb=1


100%|██████████| 18/18 [01:18<00:00,  4.34s/it]


SCRAPING Prices for https://www.sreality.cz/api/cs/v2/estates?category_type_cb=1&category_main_cb=2


100%|██████████| 22/22 [01:32<00:00,  4.22s/it]


SCRAPING Prices for https://www.sreality.cz/api/cs/v2/estates?category_type_cb=1&category_main_cb=3


...

100%|██████████| 3382/3382 [03:23<00:00, 16.61it/s]


PREPARING Estate details JSONs to DF


100%|██████████| 3382/3382 [00:50<00:00, 67.39it/s] 
c:\Users\jirka\Documents\MyProjects\Sreality\utils\utils.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  final_df.fillna("-", inplace=True)


there are already 3768 processed files


100%|██████████| 7/7 [00:04<00:00,  1.74it/s]


In [ ]:
#runner.update_geodata()
#runner.input_all_estates_to_db()
#runner.input_all_prices_to_db()
#runner.scrape_prices_and_details(["count_prodej_byty", 
#                                "count_prodej_domy"])

## Test diagnostics

In [ ]:
from diagnostics.diagnostics import Diagnostics
import json 


diag = Diagnostics()

#result = diag.summary_new_estates()

#runner.mailing.send_email(subject=f'SCRAPING SREALITY SUMMARY for {result["latest_timestamp"]}',
#                        message_text=json.dumps(result))

diag.describe_database()

## Testing requests

In [ ]:
import requests
# Just testing count
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/count"
    result =  session.request(method="GET", url=url, headers=headers).json()
    print(result)
    url_filter = f"https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=1&page=1"
    result2 =  session.request(method="GET", url=url_filter).json()
    print(result2)

In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/95413580"
    result =  session.request(method="GET", url=url, headers=headers).json()
    display(result)


## Test Utils - timestamp and mail

In [ ]:
import json
from run import Runner
runner = Runner()

In [ ]:
timestamp, _ = runner.utils.generate_timestamp()
print(timestamp)
#data_manager._insert_new_estate(df_
# new, timestamp)

In [ ]:
runner.mailing.send_email("hallo", "toto je mesidz")


In [ ]:
runner.mailing.send_email_with_attachment("hallo", "toto je mesidz", 'scraping.log')


In [ ]:
#discounts_all = runner.diag.discounts_in_last_batch(filters=None)
#runner.mailing.send_email(subject=f'SREALITY - DISCOUNTS {discounts_all["Last Date"]}',
#                        message_text=json.dumps(discounts_all))

discounts_all = runner.diag.discounts_in_last_batch(filters=None)
runner.mailing.send_email(subject=f'SREALITY - DISCOUNTS {discounts_all["Last Date"]}',
                        message_text=json.dumps(discounts_all))
        
discounts_targeted = runner.diag.discounts_in_last_batch()
runner.mailing.send_email(subject=f'SREALITY - DISCOUNTS Prodej-Byty-Praha,Středočeský {discounts_targeted["Last Date"]}',
                        message_text=json.dumps(discounts_targeted))

## Test Prepare Geo-JSONs

In [ ]:
source_folder= "c:\\Users\\jirka\\Documents\\MyProjects\\Sreality\\data_2024\\estate_details"
files = os.listdir(source_folder)
file_info = []

for file_name in files:
        file_path = os.path.join(source_folder, file_name)
        if os.path.isfile(file_path):
            size = os.path.getsize(file_path)
            file_info.append((file_name, size))

# Sort files by size
sorted_files = sorted(file_info, key=lambda x: x[1])

sorted_names = []
for file_name, size in sorted_files:
    sorted_names.append(file_name)
    print(f"{file_name}: {size} bytes")

In [ ]:
runner.geodata.enrich_jsons_with_geodata(list_of_jsons=None)

## Test table managment

In [ ]:
d = runner.data_manager.get_all_rows("price_history")
print(len(d))
d_codes = d["estate_detail"].unique()
print(len(d_codes))

In [ ]:
# price_history, estate_detail
runner.data_manager.clear_table("X")

In [ ]:
runner.data_manager.drop_table("X")

In [ ]:
runner.data_manager.create_table("price_history_v2")

## Test restrukturalizace databáze na cenu staženou na datum-range


In [ ]:

#TODO: musím to rozsekat aby to fungovalo po částech, jak kvůli testu, tak kvůli rozkouskování práce, nepřetížení systému
#TODO: musím upravit nový loading, aby nově přibyvší ceny dělaly totéž

from run import Runner
runner = Runner()

In [ ]:
df = runner.data_manager.transformation_get_old_db()
print(len(df))

In [ ]:
df_final = runner.data_manager.transformation_create_new_table(df, "price_history_new")

In [ ]:
print(len(df_final))
print(len(df_final["estate_id"].unique()))
df_final.head()

In [ ]:
# checking the estates, that have multiple prices in my sample
unique_combinations = df_final.copy() #.drop_duplicates(subset=['estate_id', 'price'])
filtered_estate_ids = unique_combinations.groupby('estate_id').filter(lambda x: x['price'].nunique() > 2)
print(len(filtered_estate_ids))
filtered_estate_ids.sort_values(by=['estate_id', "start_date"], inplace=True)
filtered_estate_ids.reset_index(drop=True)
filtered_estate_ids

In [ ]:
filtered_estate_ids[filtered_estate_ids["estate_id"]=="1012147532"]

In [ ]:
df[df["estate_id"]=="1012147532"]

In [6]:
## TEST 2 - nová fce na vkládání, do price_history_new2, pomocí insert_new_price_v2

#! check poslední scraped soubory a ulož do -kopie!
runner.input_all_prices_to_db2(limit_files=5)

# 6 dávek (42 souborů) je nyní 4
# 90 minut..  -> 1 dávka scrapingu = 80 minut...

there are already 3760 processed files
['data_prodej_byty_20241011_084220.csv', 'data_prodej_domy_20241011_084220.csv', 'data_prodej_komercni_20241011_084220.csv', 'data_prodej_ostatni_20241011_084220.csv', 'data_prodej_pozemky_20241011_084220.csv', 'data_pronajem_byty_20241010_114122.csv', 'data_pronajem_byty_20241011_011535.csv', 'data_pronajem_byty_20241011_084220.csv', 'data_pronajem_domy_20241010_114122.csv', 'data_pronajem_domy_20241011_011535.csv', 'data_pronajem_domy_20241011_084220.csv']
11


100%|██████████| 25412/25412 [37:48<00:00, 11.20it/s] 


In [4]:
runner.input_all_prices_to_db2(limit_files=5)

there are already 3755 processed files
['data_prodej_komercni_20241011_011535.csv', 'data_prodej_ostatni_20241010_114122.csv', 'data_prodej_ostatni_20241011_011535.csv', 'data_prodej_pozemky_20241010_114122.csv', 'data_prodej_pozemky_20241011_011535.csv', 'data_pronajem_byty_20241010_114122.csv', 'data_pronajem_byty_20241011_011535.csv', 'data_pronajem_domy_20241010_114122.csv', 'data_pronajem_domy_20241011_011535.csv']
9


100%|██████████| 25280/25280 [32:33<00:00, 12.94it/s]


In [ ]:
#runner.data_manager.clear_table("price_history_v1")
#runner.data_manager.drop_table("price_history_v1")
#runner.data_manager.create_table("price_history_v1")

In [ ]:

#! check poslední scraped soubory a ulož do ph1.txt a ph2.txt
runner.test_input_v1()

In [ ]:
#runner.data_manager.clear_table("price_history_v2")
#runner.data_manager.drop_table("price_history_v2")
#runner.data_manager.create_table("price_history_v2")

In [ ]:

#! check poslední scraped soubory a ulož do ph1.txt a ph2.txt
runner.test_input_v2()

## Test Scheduler

In [ ]:
import datetime as dt

from scheduler import Scheduler

score = 1

def foo():
    print(f"foo called at {dt.datetime.now()}")

schedule = Scheduler()

schedule.cyclic(dt.timedelta(seconds=6), foo)

In [ ]:
print(schedule)

In [ ]:
import time

while True:
    schedule.exec_jobs()
    time.sleep(1)

## Konec - Bonuses, exploration

In [ ]:
describe = d["id"].describe()
describe

print(describe["min"])

print(d["id"].min())

In [ ]:

# One-time - all JSONs - rename key "code" into "estate_id", so it is unified

import os
import json

folder_path = 'C://Users//X//estate_details'

json_files = [f for f in os.listdir(folder_path)]

for json_file in json_files:
    file_path = os.path.join(folder_path, json_file)
    
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    for item in data:
        if 'code' in item:
            item['estate_id'] = item.pop('code')
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

In [ ]:
# One-time - all CSVs - rename column "code" into "estate_id", so it is unified

import os
import pandas as pd

folder_path = 'C://Users//X//prices' 

csv_files = [f for f in os.listdir(folder_path)]
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    df = pd.read_csv(file_path, sep=";")
    
    df.rename(columns={'code':'estate_id'}, inplace=True)
    
    df.to_csv(file_path, sep=";", encoding="utf-8", index=False)

In [ ]:
# trying to update json object, so i can get more info for estates

#? update jde přímo přes index v listu, nebo pomocí definice itemu skrz value klíče
#? nebo přes počet klíčů. Každopádně se updatují data, ale přebývající klíče a hodnoty zůstanou!
my_list = [{"estate_id": "123","timestamp": "20240512_1234", "data": "are ok"},
           {"estate_id": "456","timestamp": "20240512_1239"}
           ]

new_object = {"estate_id": "789","timestamp": "20240512_1111",
              "some_data": "ahojky"}

#my_list[1].update(new_object)
#if item["estate_id"] == "456":

for item in my_list:
    if len(item.keys())==2:
        item.update(new_object)

display(my_list)

## Trying to visualize dates

In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import io
from datetime import datetime

estates = [
    {"id": 1, "name": "Estate A", "start_date": "2023-01-01", "end_date": "2023-03-01"},
    {"id": 2, "name": "Estate B", "start_date": "2023-02-01", "end_date": "2023-05-01"},
    {"id": 3, "name": "Estate C", "start_date": "2023-02-01", "end_date": "2023-03-01"},
    {"id": 4, "name": "Estate D", "start_date": "2023-01-01", "end_date": "2023-07-01"},
    {"id": 5, "name": "Estate E", "start_date": "2023-02-01", "end_date": "2023-05-01"},
    {"id": 6, "name": "Estate F", "start_date": "2023-01-03", "end_date": "2023-08-01"},
]

def create_estate_graph(estates):
    fig, ax = plt.subplots(figsize=(8, 3))

    estate_data = {}
    for estate in estates:
        start_date = datetime.strptime(estate["start_date"], "%Y-%m-%d")
        end_date = datetime.strptime(estate["end_date"], "%Y-%m-%d")
        key = (start_date, end_date)
        if key not in estate_data:
            estate_data[key] = 0
        estate_data[key] += 1

    # Sort data by start date and then by end date
    sorted_estate_data = sorted(estate_data.items(), key=lambda x: (x[0][0], x[0][1]), reverse = True)

    # Plot data
    for i, ((start, end), count) in enumerate(sorted_estate_data):
        ax.plot([start, end], [i, i], label=f'{count} estates', marker='o')
        ax.text(start +(end-start)/2, i, f'{count}', verticalalignment='center')

    #ax.set_yticks(range(len(sorted_estate_data)))
    #ax.set_yticklabels([f'Start: {start.strftime("%Y-%m-%d")}\nEnd: {end.strftime("%Y-%m-%d")}' for start, end in sorted_estate_data])
    ax.set_xlabel('Date')
    ax.set_title('Estate Offers Timeline')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

create_estate_graph(estates)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import io
from datetime import datetime, timedelta

estates = [
    {"id": 1, "name": "Estate A", "start_date": "2023-01-01", "end_date": "2023-03-01"},
    
    {"id": 3, "name": "Estate C", "start_date": "2023-02-01", "end_date": "2023-03-01"},
    
    {"id": 4, "name": "Estate D", "start_date": "2023-01-01", "end_date": "2023-07-01"},
    {"id": 4, "name": "Estate D", "start_date": "2023-01-01", "end_date": "2023-07-01"},
    {"id": 4, "name": "Estate D", "start_date": "2023-01-01", "end_date": "2023-07-01"},
    {"id": 4, "name": "Estate D", "start_date": "2023-01-01", "end_date": "2023-07-01"},
    {"id": 4, "name": "Estate D", "start_date": "2023-01-01", "end_date": "2023-07-01"},
    
    {"id": 2, "name": "Estate B", "start_date": "2023-02-01", "end_date": "2023-05-01"},
    {"id": 5, "name": "Estate E", "start_date": "2023-02-01", "end_date": "2023-05-01"},
    
    {"id": 6, "name": "Estate F", "start_date": "2023-01-03", "end_date": "2023-08-01"},
    {"id": 6, "name": "Estate F", "start_date": "2023-01-03", "end_date": "2023-08-01"},
    {"id": 6, "name": "Estate F", "start_date": "2023-01-03", "end_date": "2023-08-01"},
    {"id": 6, "name": "Estate F", "start_date": "2023-01-03", "end_date": "2023-08-01"},
    {"id": 6, "name": "Estate F", "start_date": "2023-01-03", "end_date": "2023-08-01"},
    {"id": 6, "name": "Estate F", "start_date": "2023-01-03", "end_date": "2023-08-01"},

]

def create_estate_graph(estates, display_position='middle'):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Process data
    estate_data = {}
    for estate in estates:
        start_date = datetime.strptime(estate["start_date"], "%Y-%m-%d")
        end_date = datetime.strptime(estate["end_date"], "%Y-%m-%d")
        key = (start_date, end_date)
        if key not in estate_data:
            estate_data[key] = 0
        estate_data[key] += 1

    # Sort data by start date and then by end date
    sorted_estate_data = sorted(estate_data.items(), key=lambda x: (x[0][0], x[0][1]), reverse=True)

    # Determine min and max counts for color scaling
    counts = [count for (_, _), count in sorted_estate_data]
    min_count = min(counts)
    max_count = max(counts)

    norm = mcolors.Normalize(vmin=min_count, vmax=max_count)
    cmap = plt.get_cmap('RdYlGn_r')  # Reversed Red to Green colormap

    # Plot data
    for i, ((start, end), count) in enumerate(sorted_estate_data):
        color = cmap(norm(count))
        ax.barh(i, (end - start).days, left=start, color=color, edgecolor='black', height = 0.8)

        # Place the count text in the middle of the bar
        mid_date = start + (end - start) / 2
        ax.text(mid_date, i, f'{count}', va='center', ha='center', color='black')


    #ax.set_yticks(range(len(sorted_estate_data)))
    #ax.set_yticklabels([f'Start: {start.strftime("%Y-%m-%d")}\nEnd: {end.strftime("%Y-%m-%d")}' for start, end in sorted_estate_data])
    ax.set_xlabel('Date')
    ax.set_title('Estate Offers Timeline')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Display count in the middle of each line
create_estate_graph(estates, display_position='middle')



In [ ]:
from tqdm import tqdm
from db_managment.data_manager import DataManager
db_manager = DataManager()

In [ ]:
df_all = db_manager.get_price_rows_for_test(1000000)
# !! první filtr - upravil jsem kod na LIMIT
# 1M za 15s, #3M za 155s, ale 5M až za 283s

print(len(df_all))
estates = df_all["estate_id"].unique()
print(len(estates))

In [ ]:
from datetime import datetime
def get_date(x):
    x = datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    return x.date()

df_all["date"] = df_all["crawled_at"].apply(get_date)
df_all = df_all[["estate_id", "date"]]

df_all.head(2)

In [ ]:
# 48s na 1k produků.
"""
res = []
for estate in tqdm(estates[-1_000:]):
    d = df_all[df_all["estate_id"] == estate]
    e = {"estate_id": estate}
    e["start"] = min(d["date"]).strftime("%Y-%m-%d")
    e["end"] = max(d["date"]).strftime("%Y-%m-%d")
    res.append(e)
"""

# 8s na 64k produktů !! -> 6*64x rychlejší = 400x speed-up
grouped = df_all.groupby("estate_id")["date"].agg(["min", "max"]).reset_index()

# Step 2: Format the dates
grouped["start"] = grouped["min"].apply(lambda x: x.strftime("%Y-%m-%d"))
grouped["end"] = grouped["max"].apply(lambda x: x.strftime("%Y-%m-%d"))

# Step 3: Convert to desired format
res = grouped[["estate_id", "start", "end"]].to_dict(orient="records")
#display(res)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import io
from datetime import datetime, timedelta

def create_estate_graph(estates):
    fig, ax = plt.subplots(figsize=(21, 12.6))

    # Process data
    estate_data = {}
    for estate in estates:
        start_date = datetime.strptime(estate["start"], "%Y-%m-%d")
        end_date = datetime.strptime(estate["end"], "%Y-%m-%d")
        key = (start_date, end_date)
        if key not in estate_data:
            estate_data[key] = 0
        estate_data[key] += 1

    # Sort data by start date and then by end date
    sorted_estate_data = sorted(estate_data.items(), key=lambda x: (x[0][0], x[0][1]), reverse=True)

    # Determine min and max counts for color scaling
    counts = [count for (_, _), count in sorted_estate_data]
    min_count = min(counts)
    max_count = max(counts)

    norm = mcolors.Normalize(vmin=min_count, vmax=max_count)
    cmap = plt.get_cmap('RdYlGn_r')  # Reversed Red to Green colormap

    # Plot data
    for i, ((start, end), count) in enumerate(sorted_estate_data):
        color = cmap(norm(count))
        ax.barh(i, (end - start).days, left=start, color=color, edgecolor='black', height = 0.8)

        # Place the count text in the middle of the bar
        mid_date = start + (end - start) / 2
        ax.text(mid_date, i, f'{count}', va='center', ha='center', color='black')


    #ax.set_yticks(range(len(sorted_estate_data)))
    #ax.set_yticklabels([f'Start: {start.strftime("%Y-%m-%d")}\nEnd: {end.strftime("%Y-%m-%d")}' for start, end in sorted_estate_data])
    ax.set_xlabel('Date')
    ax.set_title(f'Estate Offers Timeline {len(estates)}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Display count in the middle of each line
create_estate_graph(res)

